In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[7],5
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,2e-20
molecule,{'o3': 'atmpro'},o3
ng_adju,-,"[0, 0]"
ng_refs,-,"[2, 5]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.037163                    1 -0.029605
0.000750        2 -0.041619                    2 -0.033201
0.001052        3 -0.050184                    3 -0.040191
0.001476        4 -0.058309                    4 -0.046881
0.002070        5 -0.065995                    5 -0.053257
0.002904        6 -0.073232                    6 -0.059351
0.004074        7 -0.080012                    7 -0.065094
0.005714        8 -0.086320                    8 -0.070520
0.008015        9 -0.092154                    9 -0.075623
0.011243       10 -0.097618                   10 -0.080376
0.015771       11 -0.102794                   11 -0.084831
0.022122       12 -0.107767                   12 -0.088903
0.031031       13 -0.112700                   13 -0.092758
0.043528       14 -0.116676                   14 -0.095481
0.061057       15 -0.113553                   15 -0.093659
0.085645       16 -0.082259                   16 -0.071449
0.120136       17 -0.001398                   17 -0.008890
0.168516       18  0.136330                   18  0.104198
0.236378       19  0.340785                   19  0.280334
0.331549       20  0.622691                   20  0.533848
0.465100       21  0.995879                   21  0.879605
0.652400       22  1.431608                   22  1.288865
0.915100       23  1.703992                   23  1.584752
1.283650       24  1.691217                   24  1.637539
1.800600       25  1.523234                   25  1.497424
2.525700       26  1.209418                   26  1.170164
3.542800       27  0.858871                   27  0.818972
4.969550       28  0.574856                   28  0.552316
6.970850       29  0.367928                   29  0.369454
9.778100       30  0.168928                   30  0.187903
13.715850      31 -0.069323                   31 -0.052577
19.239350      32 -0.291495                   32 -0.289639
26.987250      33 -0.471295                   33 -0.491533
37.855300      34 -0.565125                   34 -0.618537
53.100050      35 -0.528503                   35 -0.598201
73.887500      36 -0.353656                   36 -0.389197
97.662500      37 -0.176670                   37 -0.184299
121.437500     38 -0.111453                   38 -0.113033
145.212500     39 -0.091896                   39 -0.092209
168.987500     40 -0.077819                   40 -0.077303
192.762500     41 -0.066675                   41 -0.065600
216.537500     42 -0.056188                   42 -0.054805
240.312500     43 -0.047134                   43 -0.045650
264.087500     44 -0.039500                   44 -0.038033
287.862500     45 -0.033213                   45 -0.031823
311.637500     46 -0.028126                   46 -0.026830
335.412500     47 -0.023761                   47 -0.022581
359.187500     48 -0.019799                   48 -0.018741
382.962500     49 -0.016298                   49 -0.015368
406.737500     50 -0.013210                   50 -0.012402
430.512500     51 -0.010311                   51 -0.009623
454.287500     52 -0.007532                   52 -0.006968
478.062500     53 -0.004862                   53 -0.004412
501.837500     54 -0.002293                   54 -0.001967
525.612500     55  0.000182                   55  0.000393
549.387500     56  0.002569                   56  0.002655
573.162500     57  0.004876                   57  0.004842
596.937500     58  0.007108                   58  0.006948
620.712500     59  0.009258                   59  0.008972
644.487500     60  0.011387                   60  0.010975
668.262500     61  0.013497                   61  0.012966
692.037500     62  0.015449                   62  0.014822
715.812500     63  0.017112                   63  0.016426
739.587500     64  0.018425                   64  0.017730
763.362500     65  0.019522                   65  0.018839
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -27.011626  0.000000 -27.011626 -26.764729 -5.373854e-08   
0.000624    2     -27.011630  0.000002 -27.011629 -26.764732  1.082314e-06   
0.000876    3     -27.011632  0.000002 -27.011630 -26.764733  1.605147e-06   
0.001229    4     -27.011636  0.000004 -27.011632 -26.764736  2.528587e-06   
0.001723    5     -27.011641  0.000006 -27.011635 -26.764740  4.103665e-06   
0.002417    6     -27.011650  0.000010 -27.011641 -26.764747  6.722770e-06   
0.003391    7     -27.011664  0.000016 -27.011649 -26.764758  1.099484e-05   
0.004757    8     -27.011688  0.000025 -27.011662 -26.764776  1.786478e-05   
0.006672    9     -27.011722  0.000041 -27.011682 -26.764803  2.878180e-05   
0.009359    10    -27.011776  0.000065 -27.011711 -26.764844  4.595847e-05   
0.013128    11    -27.011857  0.000103 -27.011754 -26.764907  7.278037e-05   
0.018415    12    -27.011980  0.000161 -27.011819 -26.765001  1.143631e-04   
0.025830    13    -27.012164  0.000251 -27.011914 -26.765144  1.784673e-04   
0.036232    14    -27.012440  0.000387 -27.012053 -26.765356  2.767515e-04   
0.050823    15    -27.012850  0.000596 -27.012254 -26.765672  4.277175e-04   
0.071291    16    -27.013489  0.000960 -27.012529 -26.766166  6.941485e-04   
0.100000    17    -27.014522  0.001714 -27.012809 -26.766968  1.253229e-03   
0.140271    18    -27.016104  0.003288 -27.012816 -26.768202  2.444751e-03   
0.196760    19    -27.018329  0.006425 -27.011903 -26.769938  4.878763e-03   
0.275997    20    -27.021136  0.012431 -27.008704 -26.772105  9.677083e-03   
0.387100    21    -27.024024  0.023515 -27.000510 -26.774241  1.883915e-02   
0.543100    22    -27.025414  0.043308 -26.982106 -26.774964  3.581915e-02   
0.761700    23    -27.021750  0.076714 -26.945036 -26.771364  6.559696e-02   
1.068500    24    -27.010755  0.127647 -26.883108 -26.761510  1.133430e-01   
1.498800    25    -26.992825  0.195922 -26.796903 -26.745642  1.809524e-01   
2.102400    26    -26.967436  0.279446 -26.687991 -26.723495  2.658832e-01   
2.949000    27    -26.938745  0.372042 -26.566704 -26.699266  3.590173e-01   
4.136600    28    -26.914296  0.468418 -26.445877 -26.680466  4.554418e-01   
5.802500    29    -26.903156  0.570719 -26.332436 -26.676291  5.602713e-01   
8.139200    30    -26.920640  0.690045 -26.230595 -26.702477  6.887317e-01   
11.417000   31    -27.000730  0.835726 -26.165003 -26.793496  8.527173e-01   
16.014700   32    -27.192647  0.989888 -26.202760 -26.998108  1.028691e+00   
22.464000   33    -27.550167  1.124715 -26.425452 -27.370171  1.179457e+00   
31.510500   34    -28.157167  1.226665 -26.930502 -28.006224  1.288717e+00   
44.200100   35    -29.060916  1.280934 -27.779983 -28.990161  1.342790e+00   
62.000000   36    -30.166662  1.272317 -28.894345 -30.241586  1.332764e+00   
85.775000   37    -31.126098  1.235745 -29.890352 -31.302025  1.296987e+00   
109.550000  38    -31.605922  1.218013 -30.387910 -31.803713  1.279578e+00   
133.325000  39    -31.917396  1.215598 -30.701797 -32.118336  1.275831e+00   
157.100000  40    -32.186420  1.225815 -30.960604 -32.385234  1.283013e+00   
180.875000  41    -32.424637  1.244871 -31.179766 -32.618091  1.298139e+00   
204.650000  42    -32.638237  1.270692 -31.367544 -32.824200  1.319477e+00   
228.425000  43    -32.827096  1.301309 -31.525788 -33.004403  1.345317e+00   
252.200000  44    -32.994006  1.335474 -31.658531 -33.162209  1.374546e+00   
275.975000  45    -33.142193  1.372418 -31.769776 -33.301235  1.406448e+00   
299.750000  46    -33.275145  1.411831 -31.863314 -33.425141  1.440722e+00   
323.525000  47    -33.396351  1.453825 -31.942527 -33.537434  1.477446e+00   
347.300000  48    -33.507822  1.498377 -32.009446 -33.640182  1.516592e+00   
371.075000  49    -33.610428  1.545222 -32.0652

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')